# Suburb Proximity Preprocessing Part 2: ORS API Data Retrieval

In this notebook the proximity data of train stations and distance to the CBD are computed.  Distance and time measures are measured by kilometers and seconds as travelled by a car on the most direct route to given the destination from the center of the suburb in question.

This notebook is divided into two sections:
1. Suburb Proximity Data Fetch
2. Suburb Proximity Exception Handling
3. Data Merging and Output

It is important to note that the centre of the suburb may not be the cultural centre of the suburb but given we do not know the address of all individual houses or have any data on population distributions within suburbs we have no way of measuring the cultural centre.  So measurements from the geographical centre to the nearest station/CBD were used instead.

In [70]:
import sys
import importlib
import json
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point

### 1 Suburb Proximity Data Fetch

In [129]:

all_suburbs_df = pd.read_csv("../../data/curated/station_data_aggregated.csv")
all_suburbs_df = all_suburbs_df.drop(columns="Unnamed: 0")
all_suburbs_df

,Suburb Cluster,SAL suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,closest_station_1_LATITUDE,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE
0,Inner Melbourne,Albert Park,20018,Albert Park-Middle Park-West St Kilda,2.8392,POLYGON ((144.9544458560001 -37.83793759699995...,0.0,POINT (144.9610525243831 -37.84517819697623),"[{'STOP_NAME': 'Flinders Street', 'LATITUDE': ...",Flinders Street,-37.818305,144.966964,Southern Cross,-37.817936,144.951411
1,Inner Melbourne,Middle Park,21677,Albert Park-Middle Park-West St Kilda,0.8475,POLYGON ((144.9613463640001 -37.84558110699993...,0.0,POINT (144.9620201791191 -37.85114538427818),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,-37.849518,144.989860,Windsor,-37.856053,144.992035
2,Inner Melbourne,St Kilda West,22345,Albert Park-Middle Park-West St Kilda,0.5290,POLYGON ((144.9701782210001 -37.85371962799996...,0.0,POINT (144.9712540823965 -37.85796076809305),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,-37.849518,144.989860,Windsor,-37.856053,144.992035
3,Inner Melbourne,Armadale,20066,Armadale,2.1430,"POLYGON ((145.024852482 -37.85092855199997, 14...",2.0,POINT (145.0202346677085 -37.85666441832981),"[{'STOP_NAME': 'Armadale', 'LATITUDE': -37.856...",Armadale,-37.856452,145.019326,Toorak,-37.850774,145.013909
4,Inner Melbourne,Carlton North,20496,Carlton North,1.8709,"POLYGON ((144.972592307 -37.79269848799998, 14...",0.0,POINT (144.9684943147982 -37.78659325732276),"[{'STOP_NAME': 'Jewell', 'LATITUDE': -37.77498...",Jewell,-37.774987,144.958717,Royal Park,-37.781193,144.952301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,Other Regional Centres,Traralgon,22569,Traralgon,56.2821,"POLYGON ((146.561066146 -38.18788998399998, 14...",1.0,POINT (146.5197809322931 -38.20334255883672),"[{'STOP_NAME': 'Traralgon', 'LATITUDE': -38.19...",Traralgon,-38.198885,146.537882,Morwell,-38.236719,146.396753
564,Other Regional Centres,Wangaratta,22680,Wanagaratta,48.7989,POLYGON ((146.3101672580001 -36.38755634399996...,1.0,POINT (146.316005335411 -36.36938362617703),"[{'STOP_NAME': 'Wangaratta', 'LATITUDE': -36.3...",Wangaratta,-36.355101,146.317038,Springhurst,-36.185893,146.470417
565,Other Regional Centres,Warragul,22698,Warragul,55.3164,POLYGON ((145.9445994800001 -38.12758481799995...,1.0,POINT (145.9299711160235 -38.15371995383722),"[{'STOP_NAME': 'Warragul', 'LATITUDE': -38.165...",Warragul,-38.165229,145.932674,Drouin,-38.136452,145.855947
566,Other Regional Centres,Warrnambool,22710,Warrnambool,65.5160,"POLYGON ((142.460390336 -38.33167768199996, 14...",2.0,POINT (142.4981786038288 -38.36885983769672),"[{'STOP_NAME': 'Warrnambool', 'LATITUDE': -38....",Warrnambool,-38.385014,142.475545,Warrnambool,-38.386392,142.538871


In [130]:
all_suburbs_df['suburb_centroid'] = all_suburbs_df['suburb_centroid'].apply(wkt.loads)

In [131]:
all_suburbs_gdf = gpd.GeoDataFrame(all_suburbs_df)

In [132]:
all_suburbs_gdf

,Suburb Cluster,SAL suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,closest_station_1_LATITUDE,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE
0,Inner Melbourne,Albert Park,20018,Albert Park-Middle Park-West St Kilda,2.8392,POLYGON ((144.9544458560001 -37.83793759699995...,0.0,POINT (144.9610525243831 -37.84517819697623),"[{'STOP_NAME': 'Flinders Street', 'LATITUDE': ...",Flinders Street,-37.818305,144.966964,Southern Cross,-37.817936,144.951411
1,Inner Melbourne,Middle Park,21677,Albert Park-Middle Park-West St Kilda,0.8475,POLYGON ((144.9613463640001 -37.84558110699993...,0.0,POINT (144.9620201791191 -37.85114538427818),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,-37.849518,144.989860,Windsor,-37.856053,144.992035
2,Inner Melbourne,St Kilda West,22345,Albert Park-Middle Park-West St Kilda,0.5290,POLYGON ((144.9701782210001 -37.85371962799996...,0.0,POINT (144.9712540823965 -37.85796076809305),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,-37.849518,144.989860,Windsor,-37.856053,144.992035
3,Inner Melbourne,Armadale,20066,Armadale,2.1430,"POLYGON ((145.024852482 -37.85092855199997, 14...",2.0,POINT (145.0202346677085 -37.85666441832981),"[{'STOP_NAME': 'Armadale', 'LATITUDE': -37.856...",Armadale,-37.856452,145.019326,Toorak,-37.850774,145.013909
4,Inner Melbourne,Carlton North,20496,Carlton North,1.8709,"POLYGON ((144.972592307 -37.79269848799998, 14...",0.0,POINT (144.9684943147982 -37.78659325732276),"[{'STOP_NAME': 'Jewell', 'LATITUDE': -37.77498...",Jewell,-37.774987,144.958717,Royal Park,-37.781193,144.952301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,Other Regional Centres,Traralgon,22569,Traralgon,56.2821,"POLYGON ((146.561066146 -38.18788998399998, 14...",1.0,POINT (146.5197809322931 -38.20334255883672),"[{'STOP_NAME': 'Traralgon', 'LATITUDE': -38.19...",Traralgon,-38.198885,146.537882,Morwell,-38.236719,146.396753
564,Other Regional Centres,Wangaratta,22680,Wanagaratta,48.7989,POLYGON ((146.3101672580001 -36.38755634399996...,1.0,POINT (146.316005335411 -36.36938362617703),"[{'STOP_NAME': 'Wangaratta', 'LATITUDE': -36.3...",Wangaratta,-36.355101,146.317038,Springhurst,-36.185893,146.470417
565,Other Regional Centres,Warragul,22698,Warragul,55.3164,POLYGON ((145.9445994800001 -38.12758481799995...,1.0,POINT (145.9299711160235 -38.15371995383722),"[{'STOP_NAME': 'Warragul', 'LATITUDE': -38.165...",Warragul,-38.165229,145.932674,Drouin,-38.136452,145.855947
566,Other Regional Centres,Warrnambool,22710,Warrnambool,65.5160,"POLYGON ((142.460390336 -38.33167768199996, 14...",2.0,POINT (142.4981786038288 -38.36885983769672),"[{'STOP_NAME': 'Warrnambool', 'LATITUDE': -38....",Warrnambool,-38.385014,142.475545,Warrnambool,-38.386392,142.538871


In [62]:
# calculate proximity to CBD and nearest train station route via car and output it to a json named "proximity_data.json"

# Define the script path directly
scripts_path = "../../scripts"
sys.path.append(scripts_path)

# Import the modules from scripts
import openrouteservice_data_download

# Reload the module to ensure the latest version is used
importlib.reload(openrouteservice_data_download)
from openrouteservice_data_download import download_ors_proximity_data

with open("../../configuration_files/configuration.json", 'r') as file:
        config = json.load(file)
api_key = config.get("API_KEY")

# assign the return dictionary just in case we need the results later
prox_dictionary = download_ors_proximity_data(all_suburbs_gdf, api_key, 'proximity_data')

Success: 200
Success: 200
Success: 200
suburb(s) no. 1 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 2 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 3 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 4 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 5 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 6 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 7 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 8 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 9 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 10 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 11 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 12 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 13 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 14 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 15 complete
Success: 200
Success: 200
Success:

### 2 Suburb Proximity Exception Handling

In [133]:
# read in recently fetched proximity data
proximity_df = pd.read_json('../../data/landing/proximity_data.json')
proximity_df = proximity_df.T

# list of missing suburbs with missing proximity data
exception_suburbs = list(proximity_df[proximity_df["time_to_CBD"].isnull()].index)

In [134]:
# column transformations for readability and consistency
proximity_df = proximity_df.reset_index()
proximity_df = proximity_df.rename(columns={"index": 'SAL_suburb'})
all_suburbs_gdf = all_suburbs_gdf.rename(columns={'SAL suburb': 'SAL_suburb'})
proximity_df

,SAL_suburb,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Albert Park,-37.845178,144.961053,4640.7,590.0,3801.0,499.7
1,Middle Park,-37.851145,144.962020,5451.7,681.5,3704.9,492.8
2,St Kilda West,-37.857961,144.971254,6547.7,772.4,2494.7,406.4
3,Armadale,-37.856664,145.020235,9160.9,963.8,520.5,106.9
4,Carlton North,-37.786593,144.968494,3661.4,431.2,2360.6,305.5
...,...,...,...,...,...,...,...
562,Traralgon,-38.203343,146.519781,161750.0,7073.1,1707.7,136.0
563,Wangaratta,-36.369384,146.316005,240991.9,10137.1,2716.7,340.3
564,Warragul,-38.153720,145.929971,104026.8,5013.4,1879.0,228.4
565,Warrnambool,-38.368860,142.498179,255755.7,11370.1,3084.6,390.3


In [135]:
# merge the proximity data to the train stations dataframe
all_suburbs_gdf = all_suburbs_gdf.merge(proximity_df[['SAL_suburb', 'distance_to_CBD','time_to_CBD', 'distance_to_station','time_to_station']], on='SAL_suburb', how='left')
all_suburbs_gdf

,Suburb Cluster,SAL_suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,closest_station_1_LATITUDE,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Inner Melbourne,Albert Park,20018,Albert Park-Middle Park-West St Kilda,2.8392,POLYGON ((144.9544458560001 -37.83793759699995...,0.0,POINT (144.9610525243831 -37.84517819697623),"[{'STOP_NAME': 'Flinders Street', 'LATITUDE': ...",Flinders Street,-37.818305,144.966964,Southern Cross,-37.817936,144.951411,4640.7,590.0,3801.0,499.7
1,Inner Melbourne,Middle Park,21677,Albert Park-Middle Park-West St Kilda,0.8475,POLYGON ((144.9613463640001 -37.84558110699993...,0.0,POINT (144.9620201791191 -37.85114538427818),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,-37.849518,144.989860,Windsor,-37.856053,144.992035,5451.7,681.5,3704.9,492.8
2,Inner Melbourne,St Kilda West,22345,Albert Park-Middle Park-West St Kilda,0.5290,POLYGON ((144.9701782210001 -37.85371962799996...,0.0,POINT (144.9712540823965 -37.85796076809305),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,-37.849518,144.989860,Windsor,-37.856053,144.992035,6547.7,772.4,2494.7,406.4
3,Inner Melbourne,Armadale,20066,Armadale,2.1430,"POLYGON ((145.024852482 -37.85092855199997, 14...",2.0,POINT (145.0202346677085 -37.85666441832981),"[{'STOP_NAME': 'Armadale', 'LATITUDE': -37.856...",Armadale,-37.856452,145.019326,Toorak,-37.850774,145.013909,9160.9,963.8,520.5,106.9
4,Inner Melbourne,Carlton North,20496,Carlton North,1.8709,"POLYGON ((144.972592307 -37.79269848799998, 14...",0.0,POINT (144.9684943147982 -37.78659325732276),"[{'STOP_NAME': 'Jewell', 'LATITUDE': -37.77498...",Jewell,-37.774987,144.958717,Royal Park,-37.781193,144.952301,3661.4,431.2,2360.6,305.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,Other Regional Centres,Traralgon,22569,Traralgon,56.2821,"POLYGON ((146.561066146 -38.18788998399998, 14...",1.0,POINT (146.5197809322931 -38.20334255883672),"[{'STOP_NAME': 'Traralgon', 'LATITUDE': -38.19...",Traralgon,-38.198885,146.537882,Morwell,-38.236719,146.396753,161750.0,7073.1,1707.7,136.0
564,Other Regional Centres,Wangaratta,22680,Wanagaratta,48.7989,POLYGON ((146.3101672580001 -36.38755634399996...,1.0,POINT (146.316005335411 -36.36938362617703),"[{'STOP_NAME': 'Wangaratta', 'LATITUDE': -36.3...",Wangaratta,-36.355101,146.317038,Springhurst,-36.185893,146.470417,240991.9,10137.1,2716.7,340.3
565,Other Regional Centres,Warragul,22698,Warragul,55.3164,POLYGON ((145.9445994800001 -38.12758481799995...,1.0,POINT (145.9299711160235 -38.15371995383722),"[{'STOP_NAME': 'Warragul', 'LATITUDE': -38.165...",Warragul,-38.165229,145.932674,Drouin,-38.136452,145.855947,104026.8,5013.4,1879.0,228.4
566,Other Regional Centres,Warrnambool,22710,Warrnambool,65.5160,"POLYGON ((142.460390336 -38.33167768199996, 14...",2.0,POINT (142.4981786038288 -38.36885983769672),"[{'STOP_NAME': 'Warrnambool', 'LATITUDE': -38....",Warrnambool,-38.385014,142.475545,Warrnambool,-38.386392,142.538871,255755.7,11370.1,3084.6,390.3


In [68]:
# filter the column where proximity data is missing
exception_suburbs_df = all_suburbs_gdf[all_suburbs_gdf["time_to_CBD"].isnull()]
print(exception_suburbs_df.shape)
exception_suburbs_df.head(5)

(35, 19)


,Suburb Cluster,SAL_suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,closest_station_1_LATITUDE,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
113,Southern Melbourne,Moorabbin Airport,21747,Cheltenham,2.9975,POLYGON ((145.0960935950001 -37.98499447799998...,0.0,POINT (145.0997750716515 -37.97742146947775),"[{'STOP_NAME': 'Parkdale', 'LATITUDE': -37.993...",Parkdale,-37.993079,145.076327,Mordialloc,-38.006583,145.087659,NaN,NaN,NaN,NaN
152,Outer Western Melbourne,Harkness,21140,Melton,16.7076,"POLYGON ((144.572960641 -37.65986148999997, 14...",0.0,POINT (144.5463356493641 -37.65451946630225),"[{'STOP_NAME': 'Melton South', 'LATITUDE': -37...",Melton South,-37.703359,144.572216,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
181,Outer Western Melbourne,Bonnie Brook,20282,Sydenham,21.3096,POLYGON ((144.6948873650001 -37.70175559899997...,0.0,POINT (144.6791032890565 -37.69417528680661),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
183,Outer Western Melbourne,Grangefields,21087,Sydenham,6.4564,POLYGON ((144.6459959920001 -37.72042481499994...,0.0,POINT (144.6364473344998 -37.7002825102328),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
186,Outer Western Melbourne,Thornhill Park,22509,Sydenham,9.5590,POLYGON ((144.6232797170001 -37.70415461399995...,1.0,POINT (144.6300205090975 -37.72757567597942),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN


In [71]:
# move suburb centroid slightly to see when the data is fetched again on the exception suburbs to see if the data can now be fetched
for row_id in exception_suburbs_df.index:
    old_point = exception_suburbs_df.loc[row_id, "suburb_centroid"]
    new_point = Point(old_point.x - 0.01, old_point.y)
    exception_suburbs_df.loc[row_id, "suburb_centroid"] = new_point
exception_suburbs_df.head(5)

,Suburb Cluster,SAL_suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,closest_station_1_LATITUDE,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
113,Southern Melbourne,Moorabbin Airport,21747,Cheltenham,2.9975,POLYGON ((145.0960935950001 -37.98499447799998...,0.0,POINT (145.0897750716515 -37.97742146947775),"[{'STOP_NAME': 'Parkdale', 'LATITUDE': -37.993...",Parkdale,-37.993079,145.076327,Mordialloc,-38.006583,145.087659,NaN,NaN,NaN,NaN
152,Outer Western Melbourne,Harkness,21140,Melton,16.7076,"POLYGON ((144.572960641 -37.65986148999997, 14...",0.0,POINT (144.5363356493641 -37.65451946630225),"[{'STOP_NAME': 'Melton South', 'LATITUDE': -37...",Melton South,-37.703359,144.572216,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
181,Outer Western Melbourne,Bonnie Brook,20282,Sydenham,21.3096,POLYGON ((144.6948873650001 -37.70175559899997...,0.0,POINT (144.6691032890565 -37.69417528680661),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
183,Outer Western Melbourne,Grangefields,21087,Sydenham,6.4564,POLYGON ((144.6459959920001 -37.72042481499994...,0.0,POINT (144.6264473344998 -37.7002825102328),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
186,Outer Western Melbourne,Thornhill Park,22509,Sydenham,9.5590,POLYGON ((144.6232797170001 -37.70415461399995...,1.0,POINT (144.6200205090975 -37.72757567597942),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN


In [72]:
exception_suburbs_df = exception_suburbs_df.reset_index()
exception_suburbs_df.head(5)

,index,Suburb Cluster,SAL_suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,closest_station_1_LATITUDE,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,113,Southern Melbourne,Moorabbin Airport,21747,Cheltenham,2.9975,POLYGON ((145.0960935950001 -37.98499447799998...,0.0,POINT (145.0897750716515 -37.97742146947775),"[{'STOP_NAME': 'Parkdale', 'LATITUDE': -37.993...",Parkdale,-37.993079,145.076327,Mordialloc,-38.006583,145.087659,NaN,NaN,NaN,NaN
1,152,Outer Western Melbourne,Harkness,21140,Melton,16.7076,"POLYGON ((144.572960641 -37.65986148999997, 14...",0.0,POINT (144.5363356493641 -37.65451946630225),"[{'STOP_NAME': 'Melton South', 'LATITUDE': -37...",Melton South,-37.703359,144.572216,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
2,181,Outer Western Melbourne,Bonnie Brook,20282,Sydenham,21.3096,POLYGON ((144.6948873650001 -37.70175559899997...,0.0,POINT (144.6691032890565 -37.69417528680661),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
3,183,Outer Western Melbourne,Grangefields,21087,Sydenham,6.4564,POLYGON ((144.6459959920001 -37.72042481499994...,0.0,POINT (144.6264473344998 -37.7002825102328),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
4,186,Outer Western Melbourne,Thornhill Park,22509,Sydenham,9.5590,POLYGON ((144.6232797170001 -37.70415461399995...,1.0,POINT (144.6200205090975 -37.72757567597942),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN


In [73]:
exception_suburbs_df = exception_suburbs_df.rename(columns={"SAL_suburb": "SAL suburb"})

In [74]:
# calculate proximity to CBD and nearest train station route via car

with open("../../configuration_files/configuration.json", 'r') as file:
        config = json.load(file)
api_key = config.get("API_KEY")

download_ors_proximity_data(exception_suburbs_df, api_key, 'exception_proximity_data')

Success: 200
Success: 200
Success: 200
suburb(s) no. 1 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 2 complete
Error: 404
{"error":{"code":2010,"message":"Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 144.6691033 -37.6941753."},"info":{"engine":{"build_date":"2024-09-13T08:01:06Z","version":"8.1.3"},"timestamp":1726959626758}}
suburb(s) no. 3 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 4 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 5 complete
Error: 404
{"error":{"code":2010,"message":"Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 144.8539380 -37.5799826."},"info":{"engine":{"build_date":"2024-09-13T08:01:06Z","version":"8.1.3"},"timestamp":1726959648824}}
suburb(s) no. 6 complete
Error: 404
{"error":{"code":2010,"message":"Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 144.7049536 -37.5470664."},"info":{"eng

{'Moorabbin Airport': {'suburb_centre_latitude': -37.97742146947775,
  'suburb_centre_longitude': 145.0897750716515,
  'distance_to_CBD': 33429.3,
  'time_to_CBD': 2787.5,
  'distance_to_station': 3166.4,
  'time_to_station': 396.1},
 'Harkness': {'suburb_centre_latitude': -37.65451946630225,
  'suburb_centre_longitude': 144.5363356493641,
  'distance_to_CBD': 51317.5,
  'time_to_CBD': 2867.7,
  'distance_to_station': 10019.8,
  'time_to_station': 901.2},
 'Bonnie Brook': {},
 'Grangefields': {'suburb_centre_latitude': -37.7002825102328,
  'suburb_centre_longitude': 144.6264473344998,
  'distance_to_CBD': 39909.4,
  'time_to_CBD': 2190.2,
  'distance_to_station': 5599.6,
  'time_to_station': 407.2},
 'Thornhill Park': {'suburb_centre_latitude': -37.72757567597942,
  'suburb_centre_longitude': 144.62002050909751,
  'distance_to_CBD': 43769.9,
  'time_to_CBD': 2428.2,
  'distance_to_station': 4016.9,
  'time_to_station': 534.2},
 'Yuroke': {},
 'Sunbury': {},
 'Wildwood': {'suburb_centre

In [75]:
# still some exceptions, repeat the same process above but move the centre in a different direction
e_proximity_df = pd.read_json('../../data/landing/exception_proximity_data.json')
e_proximity_df = e_proximity_df.T
remaining_exceptions = e_proximity_df[e_proximity_df["suburb_centre_latitude"].isnull()]

In [76]:
e_proximity_df = e_proximity_df.reset_index()
remaining_exceptions = list(e_proximity_df[e_proximity_df["suburb_centre_latitude"].isnull()]['index'])
remaining_exceptions

['Bonnie Brook',
 'Yuroke',
 'Sunbury',
 'Arthurs Creek',
 'Yan Yean',
 'Woodstock',
 'Yering',
 'Gruyere',
 'Cannons Creek',
 'Dalmore',
 'Lysterfield South',
 'Officer South',
 'Boneo',
 'Bairnsdale']

In [78]:
exception_suburbs_df = exception_suburbs_df[exception_suburbs_df["SAL suburb"].isin(remaining_exceptions)]
exception_suburbs_df.head(5)

,index,Suburb Cluster,SAL suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,closest_station_1_LATITUDE,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
2,181,Outer Western Melbourne,Bonnie Brook,20282,Sydenham,21.3096,POLYGON ((144.6948873650001 -37.70175559899997...,0.0,POINT (144.6691032890565 -37.69417528680661),"[{'STOP_NAME': 'Rockbank', 'LATITUDE': -37.729...",Rockbank,-37.729261,144.650631,Cobblebank,-37.712546,144.604108,NaN,NaN,NaN,NaN
5,223,North Western Melbourne,Yuroke,22941,Craigieburn,8.8851,"POLYGON ((144.852504143 -37.55800379799996, 14...",0.0,POINT (144.8539379952823 -37.57998257699892),"[{'STOP_NAME': 'Craigieburn', 'LATITUDE': -37....",Craigieburn,-37.601925,144.943306,Roxburgh Park,-37.638231,144.935404,NaN,NaN,NaN,NaN
6,254,North Western Melbourne,Sunbury,22391,Sunbury,132.2233,POLYGON ((144.7107941960001 -37.48235750799995...,1.0,POINT (144.7049535730173 -37.54706639426132),"[{'STOP_NAME': 'Sunbury', 'LATITUDE': -37.5792...",Sunbury,-37.579206,144.728165,Clarkefield,-37.483498,144.745373,NaN,NaN,NaN,NaN
8,267,North Eastern Melbourne,Arthurs Creek,20071,Bundoora-Greensborough-Hurstbridge,39.1753,"POLYGON ((145.238933919 -37.53735484499998, 14...",0.0,POINT (145.1968670972087 -37.56442723995832),"[{'STOP_NAME': 'Hurstbridge', 'LATITUDE': -37....",Hurstbridge,-37.639398,145.192017,Wattle Glen,-37.663968,145.181614,NaN,NaN,NaN,NaN
10,312,North Eastern Melbourne,Yan Yean,22898,Whittlesea,28.3934,"POLYGON ((145.16108122 -37.55947012899998, 145...",0.0,POINT (145.1170619234904 -37.55028218351829),"[{'STOP_NAME': 'Mernda', 'LATITUDE': -37.60254...",Mernda,-37.602549,145.100883,Hawkstowe,-37.622992,145.097396,NaN,NaN,NaN,NaN


In [79]:
for row_id in exception_suburbs_df.index:
    old_point = exception_suburbs_df.loc[row_id, "suburb_centroid"]
    new_point = Point(old_point.x + 0.02, old_point.y)
    exception_suburbs_df.loc[row_id, "suburb_centroid"] = new_point
exception_suburbs_df = exception_suburbs_df.reset_index()

# calculate proximity to CBD and nearest train station route via car

with open("../../configuration_files/configuration.json", 'r') as file:
        config = json.load(file)
api_key = config.get("API_KEY")

download_ors_proximity_data(exception_suburbs_df, api_key, 'exception_proximity_data2')

Error: 404
{"error":{"code":2010,"message":"Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 144.6891033 -37.6941753."},"info":{"engine":{"build_date":"2024-09-13T08:01:06Z","version":"8.1.3"},"timestamp":1726960316364}}
suburb(s) no. 1 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 2 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 3 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 4 complete
Error: 404
{"error":{"code":2010,"message":"Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 145.1370619 -37.5502822."},"info":{"engine":{"build_date":"2024-09-13T08:01:06Z","version":"8.1.3"},"timestamp":1726960347829}}
suburb(s) no. 5 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 6 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 7 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 8 complete
Success: 200
Success: 200
Success: 200
subur

{'Bonnie Brook': {},
 'Yuroke': {'suburb_centre_latitude': -37.57998257699892,
  'suburb_centre_longitude': 144.87393799528232,
  'distance_to_CBD': 36119.1,
  'time_to_CBD': 2430.2,
  'distance_to_station': 7238.9,
  'time_to_station': 604.2},
 'Sunbury': {'suburb_centre_latitude': -37.54706639426132,
  'suburb_centre_longitude': 144.72495357301733,
  'distance_to_CBD': 43895.6,
  'time_to_CBD': 3046.4,
  'distance_to_station': 4892.9,
  'time_to_station': 627.7},
 'Arthurs Creek': {'suburb_centre_latitude': -37.56442723995832,
  'suburb_centre_longitude': 145.2168670972087,
  'distance_to_CBD': 45597.4,
  'time_to_CBD': 3560.3,
  'distance_to_station': 12069.6,
  'time_to_station': 851.9},
 'Yan Yean': {},
 'Woodstock': {'suburb_centre_latitude': -37.54437346328859,
  'suburb_centre_longitude': 145.0711635473041,
  'distance_to_CBD': 45702.0,
  'time_to_CBD': 2934.0,
  'distance_to_station': 10568.0,
  'time_to_station': 624.6},
 'Yering': {'suburb_centre_latitude': -37.6872687796325

In [80]:
# repeat once more one final time to retrieve data for all relevant suburbs
e_proximity_df = pd.read_json('../../data/landing/exception_proximity_data2.json')
e_proximity_df = e_proximity_df.T
remaining_exceptions = e_proximity_df[e_proximity_df["suburb_centre_latitude"].isnull()]
e_proximity_df = e_proximity_df.reset_index()
remaining_exceptions = list(e_proximity_df[e_proximity_df["suburb_centre_latitude"].isnull()]['index'])
exception_suburbs_df = exception_suburbs_df[exception_suburbs_df["SAL suburb"].isin(remaining_exceptions)]

exception_suburbs_df = exception_suburbs_df.drop(columns="level_0")

In [81]:
for row_id in exception_suburbs_df.index:
    old_point = exception_suburbs_df.loc[row_id, "suburb_centroid"]
    new_point = Point(old_point.x - 0.01, old_point.y+0.01)
    exception_suburbs_df.loc[row_id, "suburb_centroid"] = new_point
exception_suburbs_df = exception_suburbs_df.reset_index()

# calculate proximity to CBD and nearest train station route via car

with open("../../configuration_files/configuration.json", 'r') as file:
        config = json.load(file)
api_key = config.get("API_KEY")

download_ors_proximity_data(exception_suburbs_df, api_key, 'exception_proximity_data3')

Success: 200
Success: 200
Success: 200
suburb(s) no. 1 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 2 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 3 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 4 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 5 complete
Success: 200
Success: 200
Success: 200
suburb(s) no. 6 complete


{'Bonnie Brook': {'suburb_centre_latitude': -37.684175286806614,
  'suburb_centre_longitude': 144.67910328905654,
  'distance_to_CBD': 41397.1,
  'time_to_CBD': 2282.3,
  'distance_to_station': 7091.6,
  'time_to_station': 499.4},
 'Yan Yean': {'suburb_centre_latitude': -37.54028218351829,
  'suburb_centre_longitude': 145.12706192349043,
  'distance_to_CBD': 51652.1,
  'time_to_CBD': 3428.2,
  'distance_to_station': 8892.9,
  'time_to_station': 815.6},
 'Dalmore': {'suburb_centre_latitude': -38.174103706978954,
  'suburb_centre_longitude': 145.42601951391754,
  'distance_to_CBD': 65742.3,
  'time_to_CBD': 3393.1,
  'distance_to_station': 12730.4,
  'time_to_station': 927.4},
 'Lysterfield South': {'suburb_centre_latitude': -37.94248311874506,
  'suburb_centre_longitude': 145.26697198690414,
  'distance_to_CBD': 36928.3,
  'time_to_CBD': 2436.7,
  'distance_to_station': 15478.4,
  'time_to_station': 1201.1},
 'Boneo': {'suburb_centre_latitude': -38.3995193954759,
  'suburb_centre_longit

### 3 Data Merging and Output

In [136]:
# read all proximity data both from exceptions and the orginial reading
prox_df = pd.read_json('../../data/landing/proximity_data.json')
prox_df = prox_df.T
prox_df = prox_df.dropna()
prox_df = prox_df.reset_index()
prox_df

,index,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Albert Park,-37.845178,144.961053,4640.7,590.0,3801.0,499.7
1,Middle Park,-37.851145,144.962020,5451.7,681.5,3704.9,492.8
2,St Kilda West,-37.857961,144.971254,6547.7,772.4,2494.7,406.4
3,Armadale,-37.856664,145.020235,9160.9,963.8,520.5,106.9
4,Carlton North,-37.786593,144.968494,3661.4,431.2,2360.6,305.5
...,...,...,...,...,...,...,...
527,Traralgon,-38.203343,146.519781,161750.0,7073.1,1707.7,136.0
528,Wangaratta,-36.369384,146.316005,240991.9,10137.1,2716.7,340.3
529,Warragul,-38.153720,145.929971,104026.8,5013.4,1879.0,228.4
530,Warrnambool,-38.368860,142.498179,255755.7,11370.1,3084.6,390.3


In [137]:
prox_df1 = pd.read_json('../../data/landing/exception_proximity_data.json')
prox_df1 = prox_df1.T
prox_df1 = prox_df1.dropna()
prox_df1 = prox_df1.reset_index()
prox_df1

,index,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Moorabbin Airport,-37.977421,145.089775,33429.3,2787.5,3166.4,396.1
1,Harkness,-37.654519,144.536336,51317.5,2867.7,10019.8,901.2
2,Grangefields,-37.700283,144.626447,39909.4,2190.2,5599.6,407.2
3,Thornhill Park,-37.727576,144.620021,43769.9,2428.2,4016.9,534.2
4,Wildwood,-37.558884,144.790164,38421.8,2794.7,10891.5,851.2
5,Cottles Bridge,-37.607083,145.213203,44251.3,3502.1,5334.8,528.2
6,Tremont,-37.879088,145.306194,39823.7,2879.2,1991.5,272.8
7,Olinda,-37.852352,145.369695,49540.3,3502.6,9049.7,784.4
8,Pearcedale,-38.197823,145.237788,61895.6,3155.4,8763.7,638.2
9,Warneet,-38.225749,145.309535,71973.2,3802.8,19011.6,1180.4


In [138]:
prox_df2 = pd.read_json('../../data/landing/exception_proximity_data2.json')
prox_df2 = prox_df2.T
prox_df2 = prox_df2.dropna()
prox_df2 = prox_df2.reset_index()
prox_df2

,index,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Yuroke,-37.579983,144.873938,36119.1,2430.2,7238.9,604.2
1,Sunbury,-37.547066,144.724954,43895.6,3046.4,4892.9,627.7
2,Arthurs Creek,-37.564427,145.216867,45597.4,3560.3,12069.6,851.9
3,Woodstock,-37.544373,145.071164,45702.0,2934.0,10568.0,624.6
4,Yering,-37.687269,145.378960,50399.9,3463.6,11714.5,825.0
5,Gruyere,-37.730106,145.475494,62763.3,5166.8,20886.2,2450.4
6,Cannons Creek,-38.221736,145.312808,71337.7,3742.1,18376.1,1119.8
7,Officer South,-38.100939,145.433048,57301.0,2940.3,4289.1,474.7


In [139]:
prox_df3 = pd.read_json('../../data/landing/exception_proximity_data3.json')
prox_df3 = prox_df3.T
prox_df3 = prox_df3.dropna()
prox_df3 = prox_df3.reset_index()
prox_df3

,index,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Bonnie Brook,-37.684175,144.679103,41397.1,2282.3,7091.6,499.4
1,Yan Yean,-37.540282,145.127062,51652.1,3428.2,8892.9,815.6
2,Dalmore,-38.174104,145.426020,65742.3,3393.1,12730.4,927.4
3,Lysterfield South,-37.942483,145.266972,36928.3,2436.7,15478.4,1201.1
4,Boneo,-38.399519,144.903622,93938.4,4445.0,31469.1,1697.0
5,Bairnsdale,-37.849836,147.587100,276630.8,12530.8,5027.9,430.7


In [140]:
# merge all the data
all_proximity_df = pd.concat([prox_df, prox_df1, prox_df2, prox_df3], axis=0)
all_proximity_df.head(5)

,index,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Albert Park,-37.845178,144.961053,4640.7,590.0,3801.0,499.7
1,Middle Park,-37.851145,144.962020,5451.7,681.5,3704.9,492.8
2,St Kilda West,-37.857961,144.971254,6547.7,772.4,2494.7,406.4
3,Armadale,-37.856664,145.020235,9160.9,963.8,520.5,106.9
4,Carlton North,-37.786593,144.968494,3661.4,431.2,2360.6,305.5


In [141]:
# check that all suburb data is in the df
all_proximity_df[all_proximity_df.isna().any(axis=1)] # should be empty

,index,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station


In [142]:
all_proximity_df = all_proximity_df.rename(columns={"index":"SAL_suburb"})
all_suburbs_gdf = all_suburbs_gdf.rename(columns={"SAL suburb":"SAL_suburb"})
print(all_proximity_df.columns)
print(all_suburbs_gdf.columns)

Index(['SAL_suburb', 'suburb_centre_latitude', 'suburb_centre_longitude',
       'distance_to_CBD', 'time_to_CBD', 'distance_to_station',
       'time_to_station'],
      dtype='object')
Index(['Suburb Cluster', 'SAL_suburb', 'SAL_CODE_2021', 'Suburb Group',
       'AREASQKM21', 'geometry', 'station_count_in_suburb', 'suburb_centroid',
       'closest_stations', 'closest_station_1_STOP_NAME',
       'closest_station_1_LATITUDE', 'closest_station_1_LONGITUDE',
       'closest_station_2_STOP_NAME', 'closest_station_2_LATITUDE',
       'closest_station_2_LONGITUDE', 'distance_to_CBD', 'time_to_CBD',
       'distance_to_station', 'time_to_station'],
      dtype='object')


In [143]:
all_proximity_df["SAL_suburb"] = all_proximity_df["SAL_suburb"].astype(str)

In [144]:
all_suburbs_gdf = all_suburbs_gdf.drop(columns=["distance_to_CBD","time_to_CBD","distance_to_station","time_to_station"])
all_suburbs_gdf.shape

(568, 15)

In [145]:
# merge the proximity data with the train station data
final_prox_df = all_suburbs_gdf.merge(all_proximity_df, on='SAL_suburb', how="left")
final_prox_df

,Suburb Cluster,SAL_suburb,SAL_CODE_2021,Suburb Group,AREASQKM21,geometry,station_count_in_suburb,suburb_centroid,closest_stations,closest_station_1_STOP_NAME,...,closest_station_1_LONGITUDE,closest_station_2_STOP_NAME,closest_station_2_LATITUDE,closest_station_2_LONGITUDE,suburb_centre_latitude,suburb_centre_longitude,distance_to_CBD,time_to_CBD,distance_to_station,time_to_station
0,Inner Melbourne,Albert Park,20018,Albert Park-Middle Park-West St Kilda,2.8392,POLYGON ((144.9544458560001 -37.83793759699995...,0.0,POINT (144.9610525243831 -37.84517819697623),"[{'STOP_NAME': 'Flinders Street', 'LATITUDE': ...",Flinders Street,...,144.966964,Southern Cross,-37.817936,144.951411,-37.845178,144.961053,4640.7,590.0,3801.0,499.7
1,Inner Melbourne,Middle Park,21677,Albert Park-Middle Park-West St Kilda,0.8475,POLYGON ((144.9613463640001 -37.84558110699993...,0.0,POINT (144.9620201791191 -37.85114538427818),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,...,144.989860,Windsor,-37.856053,144.992035,-37.851145,144.962020,5451.7,681.5,3704.9,492.8
2,Inner Melbourne,St Kilda West,22345,Albert Park-Middle Park-West St Kilda,0.5290,POLYGON ((144.9701782210001 -37.85371962799996...,0.0,POINT (144.9712540823965 -37.85796076809305),"[{'STOP_NAME': 'Prahran', 'LATITUDE': -37.8495...",Prahran,...,144.989860,Windsor,-37.856053,144.992035,-37.857961,144.971254,6547.7,772.4,2494.7,406.4
3,Inner Melbourne,Armadale,20066,Armadale,2.1430,"POLYGON ((145.024852482 -37.85092855199997, 14...",2.0,POINT (145.0202346677085 -37.85666441832981),"[{'STOP_NAME': 'Armadale', 'LATITUDE': -37.856...",Armadale,...,145.019326,Toorak,-37.850774,145.013909,-37.856664,145.020235,9160.9,963.8,520.5,106.9
4,Inner Melbourne,Carlton North,20496,Carlton North,1.8709,"POLYGON ((144.972592307 -37.79269848799998, 14...",0.0,POINT (144.9684943147982 -37.78659325732276),"[{'STOP_NAME': 'Jewell', 'LATITUDE': -37.77498...",Jewell,...,144.958717,Royal Park,-37.781193,144.952301,-37.786593,144.968494,3661.4,431.2,2360.6,305.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,Other Regional Centres,Traralgon,22569,Traralgon,56.2821,"POLYGON ((146.561066146 -38.18788998399998, 14...",1.0,POINT (146.5197809322931 -38.20334255883672),"[{'STOP_NAME': 'Traralgon', 'LATITUDE': -38.19...",Traralgon,...,146.537882,Morwell,-38.236719,146.396753,-38.203343,146.519781,161750.0,7073.1,1707.7,136.0
564,Other Regional Centres,Wangaratta,22680,Wanagaratta,48.7989,POLYGON ((146.3101672580001 -36.38755634399996...,1.0,POINT (146.316005335411 -36.36938362617703),"[{'STOP_NAME': 'Wangaratta', 'LATITUDE': -36.3...",Wangaratta,...,146.317038,Springhurst,-36.185893,146.470417,-36.369384,146.316005,240991.9,10137.1,2716.7,340.3
565,Other Regional Centres,Warragul,22698,Warragul,55.3164,POLYGON ((145.9445994800001 -38.12758481799995...,1.0,POINT (145.9299711160235 -38.15371995383722),"[{'STOP_NAME': 'Warragul', 'LATITUDE': -38.165...",Warragul,...,145.932674,Drouin,-38.136452,145.855947,-38.153720,145.929971,104026.8,5013.4,1879.0,228.4
566,Other Regional Centres,Warrnambool,22710,Warrnambool,65.5160,"POLYGON ((142.460390336 -38.33167768199996, 14...",2.0,POINT (142.4981786038288 -38.36885983769672),"[{'STOP_NAME': 'Warrnambool', 'LATITUDE': -38....",Warrnambool,...,142.475545,Warrnambool,-38.386392,142.538871,-38.368860,142.498179,255755.7,11370.1,3084.6,390.3


In [146]:
final_prox_df = final_prox_df.drop(columns=["suburb_centroid"])

In [147]:
# output the data
final_prox_df.to_csv("../../data/landing/suburb_proximity_data.csv")